# 03 — DagsHub Tracking

This notebook loads the cleaned dataset (`data/processed/survey_cleaned.csv`), rebuilds the preprocessing pipeline, trains multiple models, and logs runs & registered models to **DagsHub's MLflow**.

**Prereqs**

- You have already created `survey_cleaned.csv` in `01_eda.ipynb`.

- Place the file under `data/processed/survey_cleaned.csv`.

- You have a DagsHub repo with MLflow enabled and a Personal Access Token (PAT).



## 0) Environment & installs

In [ ]:
# If needed, install these (uncomment)
# !pip install mlflow xgboost

import os, json, tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from xgboost import XGBClassifier

import mlflow
import mlflow.sklearn
import mlflow.xgboost

print('Versions -> pandas', pd.__version__, '| sklearn loaded | mlflow', mlflow.__version__)

## 1) Configure DagsHub MLflow Tracking

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os

# Load .env from project root (works even if notebook is in notebooks/)
load_dotenv(find_dotenv())

# Read variables
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME = os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD = os.getenv("MLFLOW_TRACKING_PASSWORD")

print("URI:", MLFLOW_TRACKING_URI)
print("User:", MLFLOW_TRACKING_USERNAME)
print("Token set:", bool(MLFLOW_TRACKING_PASSWORD))

# Make sure MLflow sees them
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("price_range_models")


In [ ]:
import mlflow
print("mlflow version:", mlflow.__version__)

mlflow.set_tracking_uri(os.environ["MLFLOW_TRACKING_URI"])
mlflow.set_experiment("price_range_models")

with mlflow.start_run(run_name="ping"):
    mlflow.log_metric("ping", 1.0)
print("Run OK")


## 2) Load cleaned dataset

In [ ]:
DATA_PATH = '../data/processed/survey_cleaned.csv'  # updated path
df = pd.read_csv(DATA_PATH)
print(df.shape)
df.head()

## 3) Define preprocessing pipeline (must match 02_model_development)

In [ ]:
# Target and columns
target_col = 'price_range'
drop_cols = ['respondent_id', target_col]

X = df.drop(columns=drop_cols)
y = df[target_col]

# Columns that you wanted ordinal/label-encoding
label_cols_explicit = ['age_group', 'consume_frequency(weekly)']
label_cols_auto     = ['income_levels', 'health_concerns', 'preferable_consumption_size']

# Everything else categorical -> One-Hot Encode
all_cats = X.select_dtypes(include=['object', 'category']).columns.tolist()
nominal_cols = [c for c in all_cats if c not in (label_cols_explicit + label_cols_auto)]

# Numeric passthrough (if present)
numeric_cols = [c for c in X.columns if c not in (label_cols_explicit + label_cols_auto + nominal_cols)]

print('Ordinal explicit :', label_cols_explicit)
print('Ordinal auto     :', label_cols_auto)
print('One-hot nominal  :', nominal_cols)
print('Numeric passthru :', numeric_cols)

In [ ]:
# Ordinal categories (explicit) — adjust if your labels differ
age_group_order = ["18-25", "26-35", "36-45", "46-55", "56-70", "70+"]
consume_freq_order = ["0-2 times", "3-4 times", "5-7 times"]

ord_explicit = OrdinalEncoder(
    categories=[age_group_order, consume_freq_order],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

ord_auto = OrdinalEncoder(
    categories='auto',
    handle_unknown='use_encoded_value',
    unknown_value=-1
)

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('ord_explicit', ord_explicit, label_cols_explicit),
        ('ord_auto',     ord_auto,     label_cols_auto),
        ('onehot',       ohe,          nominal_cols),
        # numeric columns pass through via remainder
    ],
    remainder='passthrough'
)

preprocessor

## 4) Encode target and train/test split

In [ ]:
# Encode y (price_range) once for consistent classes across models
le_y = LabelEncoder()
y_enc = le_y.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.25, random_state=42, stratify=y_enc
)

print('Train:', X_train.shape, ' Test:', X_test.shape)
print('Classes:', list(le_y.classes_))

## 5) Helper functions for MLflow logging

In [ ]:
def log_confusion_matrix(y_true, y_pred, labels, title):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    fig, ax = plt.subplots()
    im = ax.imshow(cm, aspect='auto')
    ax.figure.colorbar(im, ax=ax)
    ax.set_xticks(range(len(labels))); ax.set_yticks(range(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right'); ax.set_yticklabels(labels)
    ax.set_title(title); ax.set_xlabel('Predicted'); ax.set_ylabel('True')
    fig.tight_layout()
    tmp = tempfile.NamedTemporaryFile(suffix='.png', delete=False)
    plt.savefig(tmp.name, bbox_inches='tight'); plt.close(fig)
    mlflow.log_artifact(tmp.name, artifact_path='plots')

def train_and_log(name, estimator):
    mlflow.sklearn.autolog(disable=False, log_input_examples=False)
    mlflow.xgboost.autolog(disable=False)

    with mlflow.start_run(run_name=name):
        pipe = Pipeline([('prep', preprocessor), ('model', estimator)])
        pipe.fit(X_train, y_train)

        y_pred = pipe.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        report = classification_report(
            le_y.inverse_transform(y_test),
            le_y.inverse_transform(y_pred),
            target_names=le_y.classes_,
            output_dict=True
        )

        mlflow.log_metric('accuracy', acc)
        mlflow.log_text(json.dumps(report, indent=2), 'reports/classification_report.json')
        log_confusion_matrix(y_test, y_pred, labels=list(le_y.classes_), title=f'Confusion Matrix — {name}')

        model_name = f'price-range-{name}'
        mlflow.sklearn.log_model(
            sk_model=pipe,
            artifact_path='model',
            registered_model_name=model_name
        )

        mlflow.set_tag('framework', 'sklearn')
        mlflow.set_tag('dataset', 'survey_cleaned')
        mlflow.set_tag('target', 'price_range')
        mlflow.set_tag('algo', name)

        print(f'{name}: accuracy={acc:.4f} | registered as {model_name}')

## 6) Train and log multiple models

In [ ]:
models = {
    'LogReg': LogisticRegression(max_iter=1000, n_jobs=-1),
    'RF': RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1),
    'GB': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(kernel='rbf', probability=False, random_state=42),
    'GNB': GaussianNB(),
    'XGBoost': XGBClassifier(
        n_estimators=200, learning_rate=0.1, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, n_jobs=-1, eval_metric='mlogloss'
    )
}

for name, est in models.items():
    train_and_log(name, est)

## 7) View results in DagsHub

Open your MLflow UI:
```
https://dagshub.com/<USERNAME>/<REPO>.mlflow
```
- **Experiments** → each model run with metrics and artifacts
- **Models** → each registered model (e.g., `price-range-RF`) → promote a version to Staging/Production

To serve a production model locally:
```bash
mlflow models serve -m "models:/price-range-RF@Production" -p 8080 --env-manager=local
```